![Clarify Logo](https://global-uploads.webflow.com/5e81e464dad44d3a9a32d1f4/5ed10fc3f1ff8467f4466786_logo.svg)


# Welcome to the Clarify Forecast tutorial! 📈 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/analysis_work.jpg" alt="Computer with time series data and joystick" width="400" />






# Prerequisites 
This tutorial builds up on the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/clarify/data-science-tutorials/blob/main/tutorials/Introduction.ipynb). Read the basic tutorial for more details about how to get the credentials from Clarify, read data and meta-data using the API, write data back and visualize on Clarify.


## What you need
* [Clarify](https://www.clarify.io) Account (with admin rights)
* Credential file `clarify-credentials.json` from Clarify, available to the environment runnning this notebook


# What we will do

1. [Initial setup](#init)
    - [Reading meta-data](#init_read_meta)
    - [Reading data](#init_read_data)
2. [Forecasting](#apply)
    - [Single signal forecasting](#apply-single)
3. [Write the forecast in Clarify](#write)
4. [Visualize the forecast result and collaborate with Clarify](#visualize)

---
Other resources:
* [Other tutorials on using Python with Clarify](https://github.com/clarify/data-science-tutorials/)
* [API reference](https://docs.clarify.io/reference/http)
* [SDK documentation](https://clarify.github.io/pyclarify/)
* [Merlion - time-series forecast and anomaly detection library](https://opensource.salesforce.com/Merlion/v1.0.1/tutorials.html)

# <a name="init"></a>  Initial setup
We will be using the PyClarify SDK for authentication, reading and writing signals to Clarify. The SDK is mirroring the Clarify API, thus [the reference document](https://docs.clarify.io/api/1.0/) will be a good resource if you come across any issues or want to see the capabilities of the API. 

In [ ]:
# install dependencies
!pip install requests pyclarify pandas matplotlib salesforce-merlion prophet

In [ ]:
from pyclarify import ClarifyClient
client = ClarifyClient("./clarify-credentials.json")

## <a name="init_read_meta"></a> Reading Meta data
You can retrieve the items data and meta-data from the Clarify API. This is useful in case you want to have a list of items that you have access in the script you are running. The `item_id` is necessary when retrieving data for a particular time-series from Clarify.

To obtain the result we call the method `select_items` which returns a JSON with a field `result` and sub-field `items` with a dictionary of item ids and metadata. 

In [ ]:
response = client.select_items(include_dataframe=False)
item_dict = response.result.items
for item_id, meta_data in item_dict.items():
  print(f"ID: {item_id} \t Name: {meta_data.name}")

The default behavior of the `select_items` method is to return a list of items with maximum_size defined by the API. If you want to list all items that you have access to, you can iterate over the result list and make subsequent calls to the API asking to skip an amount of items given by the `skip` parameter.

## <a name="init_read_data"></a> Reading data

Now, given the list of items that you have access to, you can choose one id of interest to retrieve data from.

In [ ]:
item_id = "c5jdi4jsbu8cohpla9vg" #change for the you item_id
starting_date = "2021-08-07T07:14:19Z" # example "2021-08-07T07:14:19Z"

# create a filter
from pyclarify import query
filter = query.Filter(fields={"id": query.In(value=[item_id])})

response = client.select_items(filter=filter, not_before=starting_date)
item_meta = list(response.result.items.values())[0]
item_name = item_meta.name
item_location = item_meta.labels['location'][0]
times = response.result.data.times
series = response.result.data.series

We proceed by converting the data from our internal `DataFrame` structure to `pandas.DataFrame` in order to use in the forecasting library. We also discard that timezone information because the forecasting library does not support timezones. The following figure shows an example graph that we obtained by running the code for plotting with a particular time-series data using pandas `df.plot` function.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/example_data_graph.png" alt="Example of time-series plotted using matplotlib" style="width: 40%;" />

In [ ]:
import pandas as pd
df = pd.DataFrame(series)
df.index = [time.replace(tzinfo=None) for time in times]
df.plot()
print(len(times))

# <a name="apply"></a> Forecasting

Given a sequence of values in a time window, we might wonder what could be the likely next values in our time-series. The task of forecasting values of a timeseries is about using the observed values until a certain point of time to predict possible future values. 

In order to do so, we will start by exploring one of the forecast models available in the library [`merlion`](https://opensource.salesforce.com/Merlion/v1.0.0/index.html). This library encapsulates multiple forecast methods, for single signals, multiple signals, allowing for modular experimentation with the algorithms, as well as composing and creating ensembles. We will only show the basic functionality here.

## <a name="apply_single"></a> Single signal forecasting



The basic elements for using the `merlion` forecasting library is the `TimeSeries` data structure, transformations `merlion.transform` applied to the data, and the configuration and forecasting model. In this case we choose to use the `Prophet` forecasting model, which means that we need to instantiate a `ProphetConfig` object, defining for example the maximum forecast steps, seasonality and transformation on the data (which is this case is the `Identity` transformation). 

In order to visualize and validade the forecast we split the original time-series data into *training* and *testing* splits. The variable `number_test_points` is used to define the number of points to be assigned for testing, while the remaining part of the split is used for training. The model is trained using only the *training* split, and then evaluated in the held-out *testing* split.

> For an in-depth tutorial of forecasting using `merlion` check the [official documentation](https://opensource.salesforce.com/Merlion/v1.0.1/examples/forecast/1_ForecastFeatures.html). You will find there information about the different models, forecasting with multiple time-series and anomaly detection.
> For more about time-series train/test splitting methods including cross validation, you can check this tutorial [time based cross validation](https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8)

In [ ]:
from merlion.utils import TimeSeries
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.transform.base import Identity
import matplotlib.pyplot as plt

number_test_points = 150

test_data = TimeSeries.from_pd(df[-number_test_points:])
train_data = TimeSeries.from_pd(df[0:-number_test_points])
config = ProphetConfig(max_forecast_steps=50, add_seasonality="auto", transform=Identity())
model  = Prophet(config)
model.train(train_data=train_data)
test_pred, test_err = model.forecast(time_stamps=test_data.time_stamps)

Finally, we collect the result of the forecast stored in the variables `test_pred` and `test_err` (the first one contains the forecast and the second one the uncertainty/error associated with the forecast). To visualize the training `train_data`, testing `test_data`, forecast and associated uncertainty, we proceed to call the function `model.plot_forecast`. The following picture is an example of a graph obtained by runnning the code in the cell below for a given time-series data. Once you run the notebook with your own time-series data you will obtain a different graph, but with a similar look and elements. 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/example_prediction_graph.png" alt="Example of the forecast result plotted using model.plot_forecast" style="width: 60%;" />


In [ ]:
# code for plotting the result of the forecat
fig, ax = model.plot_forecast(time_series_prev =train_data,time_series=test_data, plot_forecast_uncertainty=True, plot_time_series_prev=True)
plt.show()

# <a name="write"></a> Write the forecast in Clarify

We can now write back to Clary by creating DataFrames and metadata for the generated forecast and calling the method `insert` from `pyclarify`. In this case we write both the main trend of the forecast, as well as the upper and lower limit associated with the uncertainty of the forecast.

In [ ]:
from pyclarify import SignalInfo, DataFrame

config_dict = config.to_dict()
config_labels=[f"{x}:{config_dict[x]}" for x in config_dict]

def write_data_and_metadata(original_id,original_name, new_signal_id, new_name, times, values):

    new_signal_meta_data = SignalInfo(
        name=new_name, 
        description=f"Forecast for {original_name} in {item_location}",
        labels={
            "original_item_id":[original_id], 
            "number_points_testing": [number_test_points], 
            "forecast_method" :[ "Prophet"],
            "data-source": ["Forecast Tutorial"],
            "location":[item_location],
            "method_config":config_labels
        },
        engUnit="°C"
    )

    response = client.save_signals(input_ids=[new_signal_id],signals=[new_signal_meta_data])
    series = {new_signal_id : values}
    new_df = DataFrame(times=times, series=series)
    response = client.insert(new_df)
    print(response)

forecast_column_id = test_pred.names[0]
column_err = test_err.names[0]
forecast_id=forecast_column_id +"_pred"
forecast_id_upper=forecast_column_id +"_upper"
forecast_id_lower=forecast_column_id +"_lower"

forecast_values = test_pred.univariates[forecast_column_id].values
forecast_upper_values= [x+y for x,y in zip(test_pred.univariates[forecast_column_id ].values, test_err.univariates[column_err].values)]
forecast_lower_values= [x-y for x,y in zip(test_pred.univariates[forecast_column_id ].values, test_err.univariates[column_err].values)]

write_data_and_metadata(item_id, item_name, forecast_id, f"Forecast for {item_name }", times=test_pred.time_stamps, values=forecast_values)
write_data_and_metadata(item_id, item_name, forecast_id_upper, f"Forecast for {item_name } (upper bound)", times=test_err.time_stamps, values=forecast_upper_values)
write_data_and_metadata(item_id, item_name, forecast_id_lower, f"Forecast for {item_name } (lower bound)", times=test_err.time_stamps, values=forecast_lower_values)

As a result of the previous steps you should be able to find three new (or updated in case they already exist) signals under the integration that has been used in this tutorial, exemplified in the figure below.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/saved_forecast.png" alt="List of signals in Clarify" style="width: 70%;" />

# <a name="visualize"></a> Visualize the forecast result and collaborate with Clarify
<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/clarify_forecast_comment.png" alt="Visualization of forecast in the timeline in Clarify" style="width: 70%;"  />

Clarify facilitates the creation of dynamic and responsive graph visualization and collaboration around the generated forecast, for example with the possibility of creating threads of comments on a point or interval of time, as illustrated in the above figure. Once your data is written via the Clarify API you can created **items** for the forecast and the bounds of the interval characterizing the uncertainty of the forecast, as well as create customized **timelines** with your data and forecast, the details about this are beyond this tutorial. For more information about visualization and publishing signals on Clarify check the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/clarify/data-science-tutorials/blob/main/tutorials/Introduction.ipynb).

### Where to go next

* [Pattern Recognition](https://colab.research.google.com/github/clarify/data-science-tutorials/blob/main/tutorials/Pattern%20Recognition.ipynb)
* [Google Cloud Hosting](https://colab.research.google.com/github/clarify/data-science-tutorials/blob/main/tutorials/Google%20Cloud%20Hosting.ipynb)